# FaceForensics++ 다운로드

## 압축 옵션:
- **c0**: 무압축 (매우 큼, ~5TB)
- **c23**: 고품질 압축 (~500GB) 
- **c40**: 중품질 압축 (~38GB) ⭐ 추천

## 데이터 타입:
- **original**: Real 비디오
- **DeepFakes**: DeepFakes 방식
- **Face2Face**: Face2Face 방식
- **FaceSwap**: FaceSwap 방식
- **NeuralTextures**: NeuralTextures 방식
- **all**: 전체 (용량 큼)


---
## 1. Drive 마운트


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os

# Drive에 저장 (세션 끊겨도 안전)
SAVE_DIR = "/content/drive/MyDrive/FaceForensics++"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"데이터 저장 위치: {SAVE_DIR}")


---
## 2. 다운로드 스크립트 클론


In [ ]:
# download.py 직접 생성 (메일에서 받은 스크립트)
WORK_DIR = "/content/faceforensics_download"
os.makedirs(WORK_DIR, exist_ok=True)
%cd {WORK_DIR}

# 다운로드 스크립트 생성
with open('download.py', 'w') as f:
    f.write('''#!/usr/bin/env python
""" Downloads FaceForensics++ public data release """
import argparse
import os
import urllib
import urllib.request
import tempfile
import time
import sys
import json
from tqdm import tqdm
from os.path import join

FILELIST_URL = 'misc/filelist.json'
DEEPFEAKES_DETECTION_URL = 'misc/deepfake_detection_filenames.json'
DEEPFAKES_MODEL_NAMES = ['decoder_A.h5', 'decoder_B.h5', 'encoder.h5']

DATASETS = {
    'original': 'original_sequences/youtube',
    'Deepfakes': 'manipulated_sequences/Deepfakes',
    'Face2Face': 'manipulated_sequences/Face2Face',
    'FaceSwap': 'manipulated_sequences/FaceSwap',
    'NeuralTextures': 'manipulated_sequences/NeuralTextures'
}
COMPRESSION = ['raw', 'c23', 'c40']

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--output_path', type=str, required=True)
    parser.add_argument('--dataset', type=str, default='original')
    parser.add_argument('--type', type=str, default='videos')
    parser.add_argument('--c', type=str, default='c40', dest='compression')
    parser.add_argument('--num_videos', type=int, default=None)
    parser.add_argument('--server', type=str, default='EU2')
    args = parser.parse_args()
    
    if args.server == 'EU2':
        args.base_url = 'http://kaldir.vc.in.tum.de/faceforensics/v3/'
    else:
        raise Exception('Use --server EU2')
    return args

def download_file(url, out_file):
    out_dir = os.path.dirname(out_file)
    os.makedirs(out_dir, exist_ok=True)
    if not os.path.isfile(out_file):
        print(f"Downloading {url}...")
        urllib.request.urlretrieve(url, out_file)
    else:
        print(f"Skipping existing file: {out_file}")

def main():
    args = parse_args()
    dataset_path = DATASETS[args.dataset]
    c_type = args.type
    c_compression = args.compression
    
    # Get filelist
    if args.dataset == 'original':
        file_pairs = json.loads(urllib.request.urlopen(
            args.base_url + '/' + FILELIST_URL).read().decode("utf-8"))
        filelist = []
        for pair in file_pairs:
            filelist += pair
    else:
        file_pairs = json.loads(urllib.request.urlopen(
            args.base_url + '/' + FILELIST_URL).read().decode("utf-8"))
        filelist = []
        for pair in file_pairs:
            filelist.append('_'.join(pair))
            filelist.append('_'.join(pair[::-1]))
    
    if args.num_videos:
        filelist = filelist[:args.num_videos]
    
    dataset_url = args.base_url + f'{dataset_path}/{c_compression}/{c_type}/'
    output_path = join(args.output_path, dataset_path, c_compression, c_type)
    
    print(f"Downloading {len(filelist)} videos to {output_path}")
    for filename in tqdm(filelist):
        video_file = filename + '.mp4'
        download_file(dataset_url + video_file, join(output_path, video_file))

if __name__ == "__main__":
    main()
'''
)

print("✅ download.py 생성 완료!")
!ls -lh download.py


---
## 3. 다운로드 실행 (메일 승인 필요!)

### ⚠️ 용량 주의:
- **샘플 (추천)**: Real 100개 + Fake 100개 = ~2GB
- **c40 전체**: ~38GB
- **c23 전체**: ~500GB


In [ ]:
# Option 1: 샘플 다운로드 (테스트용, 빠름) ⭐ 추천
# Original 100개 + Deepfakes 100개

print("🚀 Original 비디오 다운로드 시작...")
!python download.py \
    --server EU2 \
    --c c40 \
    --dataset original \
    --type videos \
    --num_videos 100 \
    --output_path {SAVE_DIR}

print("\n🚀 Deepfakes 비디오 다운로드 시작...")
!python download.py \
    --server EU2 \
    --c c40 \
    --dataset Deepfakes \
    --type videos \
    --num_videos 100 \
    --output_path {SAVE_DIR}

print("\n✅ 샘플 다운로드 완료!")


In [ ]:
# Option 2: 더 많은 데이터 (Face2Face 추가)
# ⚠️ Option 1 대신 이거 실행

# print("🚀 Face2Face 비디오 다운로드 시작...")
# !python download.py \
#     --server EU2 \
#     --c c40 \
#     --dataset Face2Face \
#     --type videos \
#     --num_videos 100 \
#     --output_path {SAVE_DIR}

# print("✅ Face2Face 다운로드 완료!")


In [ ]:
# Option 3: 전체 다운로드 (num_videos 제한 없음, 시간 오래 걸림)
# ⚠️ 위 옵션들 대신 이거 실행

# print("🚀 전체 Original 다운로드...")
# !python download.py \
#     --server EU2 \
#     --c c40 \
#     --dataset original \
#     --type videos \
#     --output_path {SAVE_DIR}

# print("🚀 전체 Deepfakes 다운로드...")
# !python download.py \
#     --server EU2 \
#     --c c40 \
#     --dataset Deepfakes \
#     --type videos \
#     --output_path {SAVE_DIR}

# print("✅ 전체 다운로드 완료!")


---
## 4. 다운로드 확인


In [ ]:
# 다운로드된 파일 확인
!ls -lh {SAVE_DIR}
print("\n폴더별 용량:")
!du -sh {SAVE_DIR}/*


---
## 5. 파인튜닝 노트북에서 사용하기

다운로드 완료 후 `finetune_wilddeepfake.ipynb`로 이동하여:

```python
# 셀 12번 수정:
VIDEO_DIRS = {
    'real': "/content/drive/MyDrive/FaceForensics++/original_sequences/youtube/c40/videos",
    'fake': "/content/drive/MyDrive/FaceForensics++/manipulated_sequences/DeepFakes/c40/videos"
}
```

### 추천 전략:
1. **샘플로 먼저 테스트** (Option 1)
2. Val F1 0.70 이상 나오면
3. 전체 데이터로 재학습 (Option 2 or 3)

### 여러 manipulation 앙상블:
```python
# DeepFakes + Face2Face + FaceSwap 섞어서 학습
VIDEO_DIRS_FAKE = [
    f"{SAVE_DIR}/manipulated_sequences/DeepFakes/c40/videos",
    f"{SAVE_DIR}/manipulated_sequences/Face2Face/c40/videos",
    f"{SAVE_DIR}/manipulated_sequences/FaceSwap/c40/videos"
]
```
